# NumPy Strings Module (NumPy 2.0+)

The `numpy.strings` module provides fast, vectorized string operations for NumPy arrays.

**Key changes from numpy.char:**
- `numpy.char` is deprecated (still works but won't get updates)
- `numpy.strings` provides ufuncs with better performance
- New `StringDType` for variable-length strings (no padding waste)
- New `np.strings.slice()` function added in NumPy 2.3

## Setup

In [28]:
import numpy as np
from numpy.dtypes import StringDType
import numba

print(f"Numba version: {numba.__version__}")
print(f"NumPy version: {np.__version__}")

Numba version: 0.63.1
NumPy version: 2.3.4


## String Data Types

NumPy has three string dtype options:

| Type | Description | Use Case |
|------|-------------|----------|
| `numpy.str_` (U) | Fixed-width Unicode | Known max length, memory-mapped files |
| `numpy.bytes_` (S) | Fixed-width bytes | Binary data, ASCII only |
| `StringDType` | Variable-width UTF-8 | Unknown lengths, modern workflows |

In [2]:
# Fixed-width Unicode (legacy) - all strings padded to same size
fixed = np.array(["hello", "world", "numpy"])
print(f"Fixed-width: {fixed}")
print(f"dtype: {fixed.dtype}  (U5 = 5 Unicode chars max)")
print(f"itemsize: {fixed.dtype.itemsize} bytes per element")

Fixed-width: ['hello' 'world' 'numpy']
dtype: <U5  (U5 = 5 Unicode chars max)
itemsize: 20 bytes per element


In [3]:
# Variable-width StringDType (new in NumPy 2.0)
# No padding - stores exactly what's needed
variable = np.array(["short", "this is a much longer string", "x"], dtype=StringDType())
print(f"Variable-width: {variable}")
print(f"dtype: {variable.dtype}")

Variable-width: ['short' 'this is a much longer string' 'x']
dtype: StringDType()


In [4]:
# StringDType with missing value support
dt_with_na = StringDType(na_object=None)
with_missing = np.array(["hello", None, "world"], dtype=dt_with_na)
print(f"With missing: {with_missing}")
print(f"Is None? {with_missing[1] is None}")

With missing: ['hello' None 'world']
Is None? True


In [5]:
# StringDType with NaN as missing value (propagates like numeric NaN)
dt_with_nan = StringDType(na_object=np.nan)
with_nan = np.array(["hello", np.nan, "world"], dtype=dt_with_nan)
print(f"With NaN: {with_nan}")
print(f"Concatenated: {with_nan + with_nan}")  # NaN propagates

With NaN: ['hello' nan 'world']
Concatenated: ['hellohello' nan 'worldworld']


## String Slicing (np.strings.slice)

New in NumPy 2.3 - fast native string slicing with full Python slice semantics.

In [6]:
arr = np.array(["hello", "world", "numpy", "strings"])

# Note: np.strings.slice(arr, x) with one arg means stop=x (like s[:x])
# To get s[x:], you need np.strings.slice(arr, x, None)

# Slice first N characters (like s[:2])
print(f"First 2 chars (s[:2]):  {np.strings.slice(arr, 2)}")

# Slice from position N (like s[2:])
print(f"From pos 2 (s[2:]):     {np.strings.slice(arr, 2, None)}")

# Slice with start:stop (like s[1:4])
print(f"Positions 1-4 (s[1:4]): {np.strings.slice(arr, 1, 4)}")

# Slice with step (like s[::2])
print(f"Every 2nd (s[::2]):     {np.strings.slice(arr, None, None, 2)}")

First 2 chars (s[:2]):  ['he' 'wo' 'nu' 'st']
From pos 2 (s[2:]):     ['he' 'wo' 'nu' 'st']
Positions 1-4 (s[1:4]): ['ell' 'orl' 'ump' 'tri']
Every 2nd (s[::2]):     ['hlo' 'wrd' 'nmy' 'srns']


In [7]:
# Negative indices work too
arr = np.array(["hello world", "numpy arrays", "python code"])

print(f"Last 5 chars:  {np.strings.slice(arr, -5, None)}")
print(f"All but last:  {np.strings.slice(arr, None, -1)}")
print(f"Reversed:      {np.strings.slice(arr, None, None, -1)}")

Last 5 chars:  ['hello ' 'numpy a' 'python']
All but last:  ['hello worl' 'numpy array' 'python cod']
Reversed:      ['dlrow olleh' 'syarra ypmun' 'edoc nohtyp']


In [8]:
# Different slice parameters per element (broadcasting)
arr = np.array(["abcdefgh", "12345678", "ABCDEFGH"])
starts = np.array([0, 2, 4])
stops = np.array([3, 5, 7])

print(f"Original:     {arr}")
print(f"Variable slice: {np.strings.slice(arr, starts, stops)}")

Original:     ['abcdefgh' '12345678' 'ABCDEFGH']
Variable slice: ['abc' '345' 'EFG']


In [9]:
# Works with Unicode (character-based, not byte-based)
unicode_arr = np.array(["hello world", "γεία σου", "你好世界", "👋🌍🎉"], 
                        dtype=StringDType())

print(f"Original:    {unicode_arr}")
print(f"First 2:     {np.strings.slice(unicode_arr, 2)}")
print(f"Reversed:    {np.strings.slice(unicode_arr, None, None, -1)}")

Original:    ['hello world' 'γεία σου' '你好世界' '👋🌍🎉']
First 2:     ['he' 'γε' '你好' '👋🌍']
Reversed:    ['dlrow olleh' 'υοσ αίεγ' '界世好你' '🎉🌍👋']


## String Operations

In [10]:
# Concatenation
a = np.array(["hello", "good"])
b = np.array([" world", "bye"])

print(f"add (concat): {np.strings.add(a, b)}")
print(f"multiply:     {np.strings.multiply(a, 3)}")

add (concat): ['hello world' 'goodbye']
multiply:     ['hellohellohello' 'goodgoodgood']


In [11]:
# Case operations
arr = np.array(["Hello World", "NUMPY", "python"])

print(f"Original:   {arr}")
print(f"upper:      {np.strings.upper(arr)}")
print(f"lower:      {np.strings.lower(arr)}")
print(f"capitalize: {np.strings.capitalize(arr)}")
print(f"title:      {np.strings.title(arr)}")
print(f"swapcase:   {np.strings.swapcase(arr)}")

Original:   ['Hello World' 'NUMPY' 'python']
upper:      ['HELLO WORLD' 'NUMPY' 'PYTHON']
lower:      ['hello world' 'numpy' 'python']
capitalize: ['Hello world' 'Numpy' 'Python']
title:      ['Hello World' 'Numpy' 'Python']
swapcase:   ['hELLO wORLD' 'numpy' 'PYTHON']


In [12]:
# Strip whitespace/characters
arr = np.array(["  hello  ", "__world__", "xxpythonxx"])

print(f"Original: {repr(arr)}")
print(f"strip:    {np.strings.strip(arr)}")
print(f"lstrip:   {np.strings.lstrip(arr)}")
print(f"rstrip:   {np.strings.rstrip(arr)}")
print(f"strip(_): {np.strings.strip(arr, '_')}")
print(f"strip(x): {np.strings.strip(arr, 'x')}")

Original: array(['  hello  ', '__world__', 'xxpythonxx'], dtype='<U10')
strip:    ['hello' '__world__' 'xxpythonxx']
lstrip:   ['hello  ' '__world__' 'xxpythonxx']
rstrip:   ['  hello' '__world__' 'xxpythonxx']
strip(_): ['  hello  ' 'world' 'xxpythonxx']
strip(x): ['  hello  ' '__world__' 'python']


In [13]:
# Justification and padding
arr = np.array(["hi", "hello", "x"])

print(f"ljust(10):    {np.strings.ljust(arr, 10, '.')}")
print(f"rjust(10):    {np.strings.rjust(arr, 10, '.')}")
print(f"center(10):   {np.strings.center(arr, 10, '.')}")
print(f"zfill(5):     {np.strings.zfill(arr, 5)}")

ljust(10):    ['hi........' 'hello.....' 'x.........']
rjust(10):    ['........hi' '.....hello' '.........x']
center(10):   ['....hi....' '..hello...' '....x.....']
zfill(5):     ['000hi' 'hello' '0000x']


## Find and Replace

In [14]:
# Replace substrings
arr = np.array(["hello world", "world of numpy", "hello hello"])

print(f"Original:        {arr}")
print(f"replace all:     {np.strings.replace(arr, 'hello', 'hi')}")
print(f"replace first 1: {np.strings.replace(arr, 'hello', 'hi', 1)}")

Original:        ['hello world' 'world of numpy' 'hello hello']
replace all:     ['hi world' 'world of numpy' 'hi hi']
replace first 1: ['hi world' 'world of numpy' 'hi hello']


In [15]:
# Find substrings (returns -1 if not found)
arr = np.array(["hello world", "numpy arrays", "python code"])

print(f"Original:      {arr}")
print(f"find 'o':      {np.strings.find(arr, 'o')}")
print(f"rfind 'o':     {np.strings.rfind(arr, 'o')}")
print(f"count 'o':     {np.strings.count(arr, 'o')}")

Original:      ['hello world' 'numpy arrays' 'python code']
find 'o':      [ 4 -1  4]
rfind 'o':     [ 7 -1  8]
count 'o':     [2 0 2]


In [16]:
# Partition around separator
arr = np.array(["user@domain.com", "name@example.org", "test@test.net"])

before, sep, after = np.strings.partition(arr, '@')
print(f"Original: {arr}")
print(f"Before @: {before}")
print(f"After @:  {after}")

Original: ['user@domain.com' 'name@example.org' 'test@test.net']
Before @: ['user' 'name' 'test']
After @:  ['domain.com' 'example.org' 'test.net']


## String Comparisons

In [17]:
# Element-wise comparison
a = np.array(["apple", "banana", "cherry"])
b = np.array(["apple", "BANANA", "cherry"])

print(f"a: {a}")
print(f"b: {b}")
print(f"equal:         {np.strings.equal(a, b)}")
print(f"not_equal:     {np.strings.not_equal(a, b)}")
print(f"greater:       {np.strings.greater(a, b)}")
print(f"less:          {np.strings.less(a, b)}")

a: ['apple' 'banana' 'cherry']
b: ['apple' 'BANANA' 'cherry']
equal:         [ True False  True]
not_equal:     [False  True False]
greater:       [False  True False]
less:          [False False False]


In [18]:
# Prefix/suffix testing
arr = np.array(["hello.py", "world.txt", "numpy.py", "data.csv"])

print(f"Original:     {arr}")
print(f"endswith .py: {np.strings.endswith(arr, '.py')}")
print(f"startswith h: {np.strings.startswith(arr, 'h')}")

Original:     ['hello.py' 'world.txt' 'numpy.py' 'data.csv']
endswith .py: [ True False  True False]
startswith h: [ True False False False]


## String Information

In [19]:
# String length
arr = np.array(["hello", "a", "numpy strings", ""])

print(f"Original: {arr}")
print(f"str_len:  {np.strings.str_len(arr)}")

Original: ['hello' 'a' 'numpy strings' '']
str_len:  [ 5  1 13  0]


In [20]:
# Character type tests
arr = np.array(["hello", "123", "Hello123", "HELLO", "  ", ""])

print(f"Original:   {arr}")
print(f"isalpha:    {np.strings.isalpha(arr)}")
print(f"isdigit:    {np.strings.isdigit(arr)}")
print(f"isalnum:    {np.strings.isalnum(arr)}")
print(f"islower:    {np.strings.islower(arr)}")
print(f"isupper:    {np.strings.isupper(arr)}")
print(f"isspace:    {np.strings.isspace(arr)}")

Original:   ['hello' '123' 'Hello123' 'HELLO' '  ' '']
isalpha:    [ True False False  True False False]
isdigit:    [False  True False False False False]
isalnum:    [ True  True  True  True False False]
islower:    [ True False False False False False]
isupper:    [False False False  True False False]
isspace:    [False False False False  True False]


## String Formatting

In [21]:
# String formatting with mod (% operator)
# Each template is formatted with corresponding value
templates = np.array(["Hello %s!", "Count: %d", "Name: %s"])
values = np.array(["World", 42, "NumPy"], dtype=object)

# mod applies % operator element-wise
result = np.strings.mod(templates, values)
print(f"Formatted: {result}")

Formatted: ['Hello World!' 'Count: 42' 'Name: NumPy']


In [22]:
# Expand tabs
arr = np.array(["col1\tcol2\tcol3", "a\tb\tc"])

print("Original (with tabs):")
for s in arr:
    print(f"  {repr(s)}")

print("\nExpanded (tabsize=8):")
for s in np.strings.expandtabs(arr, 8):
    print(f"  '{s}'")

Original (with tabs):
  np.str_('col1\tcol2\tcol3')
  np.str_('a\tb\tc')

Expanded (tabsize=8):
  'col1    col2    col3'
  'a       b       c'


## Encoding/Decoding

In [23]:
# Encode strings to bytes
arr = np.array(["hello", "world", "你好"])
encoded = np.strings.encode(arr, 'utf-8')
print(f"Original: {arr}")
print(f"Encoded:  {encoded}")

# Decode bytes to strings
decoded = np.strings.decode(encoded, 'utf-8')
print(f"Decoded:  {decoded}")

Original: ['hello' 'world' '你好']
Encoded:  [b'hello' b'world' b'\xe4\xbd\xa0\xe5\xa5\xbd']
Decoded:  ['hello' 'world' '你好']


## Performance: np.strings vs Python loops

In [24]:
import timeit

# Generate test data
n = 100_000
data = np.array([f"string_{i:05d}" for i in range(n)])

def benchmark(name, func, runs=5):
    t = min(timeit.repeat(func, number=1, repeat=runs)) * 1000
    print(f"{name:25s}: {t:8.2f}ms")

print(f"Benchmarks with {n:,} strings:")
print("-" * 40)

# Replace first character with 'A'
benchmark("np.strings.slice + add", 
          lambda: np.strings.add('A', np.strings.slice(data, 1, None)))

benchmark("Python list comprehension", 
          lambda: ['A' + s[1:] for s in data])

print()

# Upper case
benchmark("np.strings.upper", 
          lambda: np.strings.upper(data))

benchmark("Python list comprehension", 
          lambda: [s.upper() for s in data])

print()

# Find substring
benchmark("np.strings.find", 
          lambda: np.strings.find(data, '_'))

benchmark("Python list comprehension", 
          lambda: [s.find('_') for s in data])

Benchmarks with 100,000 strings:
----------------------------------------
np.strings.slice + add   :     3.49ms
Python list comprehension:    16.55ms

np.strings.upper         :    13.33ms
Python list comprehension:    11.69ms

np.strings.find          :     0.45ms
Python list comprehension:    11.50ms


## Summary

### Key Functions in numpy.strings

| Category | Functions |
|----------|----------|
| **Slicing** | `slice` |
| **Concatenation** | `add`, `multiply` |
| **Case** | `upper`, `lower`, `capitalize`, `title`, `swapcase` |
| **Whitespace** | `strip`, `lstrip`, `rstrip`, `expandtabs` |
| **Padding** | `ljust`, `rjust`, `center`, `zfill` |
| **Find/Replace** | `find`, `rfind`, `index`, `rindex`, `count`, `replace` |
| **Split** | `partition`, `rpartition` |
| **Comparison** | `equal`, `not_equal`, `greater`, `less`, `greater_equal`, `less_equal` |
| **Tests** | `startswith`, `endswith`, `isalpha`, `isdigit`, `isalnum`, `islower`, `isupper`, `isspace`, `isnumeric`, `isdecimal`, `istitle` |
| **Info** | `str_len` |
| **Encoding** | `encode`, `decode` |
| **Format** | `mod`, `translate` |

### When to Use What

| Scenario | Recommendation |
|----------|---------------|
| Variable-length strings | `StringDType()` |
| Missing value support | `StringDType(na_object=None)` or `StringDType(na_object=np.nan)` |
| Fixed-width, memory-mapped | `np.str_` (U dtype) |
| Vectorized operations | `np.strings.*` functions |
| Single string manipulation | Python str methods |